The important cells in this notebook were updated after changing the flow code to minimize relative conductance. However, not all cells have been re-run, so there may be old results in here.

In [ ]:
import localgraphclustering as lgc
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.patches import PathPatch
# shapefile names 
riverslakes_shape='ne_50m_rivers_lake_centerlines_scale_rank'
lakes_shape='ne_50m_lakes'


In [ ]:
# I don't understand why, but this needs to be in it's own cell...
# plt.rcParams['figure.dpi'] = 300
# Only run this if you want high-quality intermediate figures -- but they get BIG! 

In [ ]:
g = lgc.GraphLocal()
g.read_graph("usroads-simplify.edges", separator="\t")
xy = np.loadtxt("usroads-simplify.xy")
g.draw(xy,nodesize=0.5,edgealpha=0.5,linewidth=0.5)

In [ ]:
nodecolor='#fc8d62'
hicolor='#8da0cb'
def drawusmap():
  f,axes = plt.subplots(1,1, figsize=(10,10))
  axes.set_rasterization_zorder(5)
  axes.xaxis.set_major_locator(plt.NullLocator())
  axes.yaxis.set_major_locator(plt.NullLocator())

  # setup Lambert Conformal basemap.
  # set resolution=None to skip processing of boundary datasets.
  # from https://github.com/matplotlib/basemap/blob/master/examples/fillstates.py
  m = Basemap(llcrnrlon=-119.25,llcrnrlat=19.75,urcrnrlon=-63.5,urcrnrlat=49,
              projection='lcc',lat_1=33,lat_2=45,lon_0=-95,epsg=2163)
  m.shadedrelief(alpha=0.75,ax=axes)
  xlims = axes.get_xlim()
  ylims = axes.get_ylim()
  #m.arcgisimage(service = 'World_Shaded_Relief', xpixels = 12000, alpha=0.5)
  m.drawcountries(ax=axes,color='grey',linewidth=0.5,zorder=0.5)
  m.drawstates(ax=axes,color='grey',linewidth=0.5,zorder=0.5)
  
  # handle Rivers
  m.readshapefile(riverslakes_shape,'myrivers',color='mediumblue',
                  linewidth=1.25,zorder=2, drawbounds=True)

  # handle Lakes
  m.readshapefile(lakes_shape, 'mylakes',drawbounds=False)
  patches = []
  for info, shape in zip(m.mylakes_info, m.mylakes):
      patches.append( Polygon(np.array(shape), True) )
  axes.add_collection(PatchCollection(patches, 
                                      facecolor='dodgerblue',
                                      edgecolor='grey', 
                                      linewidths=0.5, zorder=0.75))

  # draw graph
  xp,yp = m(xy[:,0],xy[:,1])
  xyp = np.column_stack((xp,yp))
  #drawing = g.draw(xyp,nodesize=0.8,edgealpha=0.5,linewidth=0.5,
                   #nodealpha=0.5,axs=axes,nodecolor=nodecolor)
  drawing = g.draw(xyp,nodesize=1.0,edgealpha=0.5,linewidth=0.8,
                   nodealpha=0.5,axs=axes,nodecolor=nodecolor)
  drawing.nodewidth(range(g._num_vertices), 0.2)
  
  #axes.set_xlim(np.min(xp)-100000,np.max(xp)+20000)
  axes.set_xlim(xlims)
  axes.set_ylim(np.min(yp)-50000,np.max(yp)+50000)
  
  return drawing, m, axes
drawing,bm,ax = drawusmap()
print(ax.get_xlim())
# test zooming
plt.savefig('usroads-simplify-network.png', dpi=200,bbox_inches='tight',pad_inches = 0)


In [ ]:
drawing,bm,ax = drawusmap()
# Test zooming 
ax.set_xlim([2*164014,5*164014])
ax.set_ylim([4*316652,6*316652])
#m = Basemap(llcrnrlon=-119,llcrnrlat=40,urcrnrlon=-100,urcrnrlat=50,
#              projection='lcc',lat_1=33,lat_2=45,lon_0=-95, epsg=2163)
# need to add this for zooming. 
bm.arcgisimage(xpixels=4000, alpha=0.75, axes=ax, zorder=0.5, service='World_Shaded_Relief')

In [ ]:
# Setup set highlighting 
def myhighlight(drawing,S,reverse=False):
  Sset = set(S)
  Sbar = set(range(drawing.G._num_vertices)) - Sset
  if reverse:
    S = list(Sbar)
    Sset,Sbar = Sbar,Sset
  drawing.nodecolor(S, '#7570b3', alpha=0.25)
  drawing.nodecolor(list(Sbar), alpha=0.25)
  lws = np.asarray(drawing.edge_collection.get_linewidths())
  if len(lws) == 1:
    lws = np.array([lws[0]]*len(drawing.edge_mapping))
  for (i,j) in drawing.edge_mapping.keys():
    if (i in Sbar and j in Sbar) or (i in Sset and j in Sset):
      drawing.edgecolor(i,j,alpha=0.33)
    else:
      drawing.edgecolor(i,j,c='crimson',alpha=1.0)
      idx = drawing.edge_mapping[(i,j)]
      lws[idx] = 1.25
  drawing.edge_collection.set_linewidths(lws)

  
S1 = [i for i in range(g._num_vertices) if xy[i,1] >= 39]  
drawing, bm, ax = drawusmap()
myhighlight(drawing, S1)  
plt.savefig('usroads-simplify-network-1.png', dpi=200,bbox_inches='tight',pad_inches=0)

In [ ]:
S1small = lgc.flow_clustering(g, S1, method='mqi')[0] # make a small set

## Try simple sets

In [ ]:
import pprint 
# Return the smaller of Si or Si reversed
def simplify_set(g, Si):
  stats = g.set_scores(Si)
  if stats["voleff"] != stats["voltrue"]:
    # reverse 
    Si = list(set(range(drawing.G._num_vertices)) - set(Si))
  return Si

def find_boundary(g,Si):
    Si_s = np.array(list(set(range(g._num_vertices)).difference(Si)))
    Si,Si_s = set(Si),set(Si_s)
    boundary = set()
    for i in Si:
      for k in range(g.ai[i],g.ai[i+1]):
        if g.aj[k] in Si_s:
          boundary.add(i)
          boundary.add(g.aj[k])
    return list(boundary)
  
def bigger_graph(drawing):
  drawing.nodes_collection.set_sizes(np.asarray(drawing.nodes_collection.get_sizes())*3)
  drawing.edge_collection.set_linewidths(np.asarray(drawing.edge_collection.get_linewidths())*1.25)
  
  
def zoomfig_area(g, Si, bm, axes, offset=0.05):
    xlims = axes.get_xlim()
    ylims = axes.get_ylim()
    original_area = (xlims[1] - xlims[0])*(ylims[1]-ylims[0])
    
    xp, yp = bm(xy[:,0], xy[:,1]) # translate coordinates 
    #xp = xy[:,0] # we now translate these later incase we want to get a better arcgis image
    #yp = xy[:,1]
    Si = simplify_set(g, Si)
    ss = g.set_scores(Si)
    boundary = find_boundary(g,Si)
    
    xlim = (np.min(xp[boundary]),np.max(xp[boundary]))
    ylim = (np.min(yp[boundary]),np.max(yp[boundary]))
    
    local = False
    big = False
    if ss["voleff"] <= g.vol_G/5: # also include the set and show it 
        xlim = (min(np.min(xp[Si]), xlim[0]),max(np.max(xp[Si]), xlim[1]))
        ylim = (min(np.min(yp[Si]), ylim[0]),max(np.max(yp[Si]), ylim[1]))
        # and change the image to 
        #bm.arcgisimage(service = 'World_Shaded_Relief', xpixels = 2000, alpha=0.75)
        local = True
        new_area = (xlim[1]-xlim[0])*(ylim[1]-ylim[0])
        if new_area/original_area <= 0.2:
          big = True
        
    xlim = (max(xlim[0]-(xlim[1]-xlim[0])*offset,xlims[0]), min(xlim[1]+(xlim[1]-xlim[0])*offset,xlims[1]))
    ylim = (max(ylim[0]-(ylim[1]-ylim[0])*offset,ylims[0]), min(ylim[1]+(ylim[1]-ylim[0])*offset,ylims[1]))
    return xlim, ylim, local, big
  
def zoomfig(g,Si,bm,axes,drawing,offset=0.05):
    
    # get the initial figure size
    xlims = axes.get_xlim()
    ylims = axes.get_ylim()
    original_area = (xlims[1] - xlims[0])*(ylims[1]-ylims[0])
    
    xp, yp = bm(xy[:,0], xy[:,1]) # translate coordinates 
    #xp = xy[:,0] # we now translate these later incase we want to get a better arcgis image
    #yp = xy[:,1]
    Si = simplify_set(g, Si)
    ss = g.set_scores(Si)
    boundary = find_boundary(g,Si)
    
    xlim = (np.min(xp[boundary]),np.max(xp[boundary]))
    ylim = (np.min(yp[boundary]),np.max(yp[boundary]))
    
    if ss["voleff"] <= g.vol_G/5: # also include the set and show it 
        xlim = (min(np.min(xp[Si]), xlim[0]),max(np.max(xp[Si]), xlim[1]))
        ylim = (min(np.min(yp[Si]), ylim[0]),max(np.max(yp[Si]), ylim[1]))
        # and change the image to 
        #bm.arcgisimage(service = 'World_Shaded_Relief', xpixels = 2000, alpha=0.75)
        bm.arcgisimage(xpixels=5000, alpha=0.75, axes=axes, 
                        zorder=0.5, service='World_Shaded_Relief')
        new_area = (xlim[1]-xlim[0])*(ylim[1]-ylim[0])
        if new_area/original_area <= 0.2:
          bigger_graph(drawing)
        
        
    xlim = (max(xlim[0]-(xlim[1]-xlim[0])*offset,xlims[0]), min(xlim[1]+(xlim[1]-xlim[0])*offset,xlims[1]))
    ylim = (max(ylim[0]-(ylim[1]-ylim[0])*offset,ylims[0]), min(ylim[1]+(ylim[1]-ylim[0])*offset,ylims[1]))
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    
def show_set_figures(g, S, name, zoom):
  pprint.pprint(g.set_scores(S))
  drawing, bm, ax = drawusmap()
  myhighlight(drawing, S)
  plt.savefig('%s.png'%(name), dpi=200,bbox_inches='tight',pad_inches = 0)
  
  if zoom: # figure out the new area, and save another marked image
    xlim,ylim,local,big = zoomfig_area(g, S, bm, ax)
    ax.add_patch(Polygon([(xlim[0],ylim[0]),(xlim[0],ylim[1]),(xlim[1],ylim[1]),(xlim[1],ylim[0])], 
                         closed=True, fc='none', edgecolor='k', lw=1.75, zorder=2))
    plt.savefig('%s-region.png'%(name), dpi=200,bbox_inches='tight',pad_inches = 0)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if local:
      bm.arcgisimage(xpixels=5000, alpha=0.75, axes=ax, 
                        zorder=0.5, service='World_Shaded_Relief')
      if big:
        bigger_graph(drawing)
    #zoomfig(g, S, bm, ax, drawing)
    plt.savefig('%s-zoom.png'%(name), dpi=200,bbox_inches='tight',pad_inches = 0)
  
def study_simple_set(g, Si, name, mqi=True, fi=True, sl01=True, sl05=True, sl1=True, zoom=True):
    Si = simplify_set(g, Si)
    result = {'input':Si}
    print("Input set: ")
    show_set_figures(g, Si, 'usroads-simplify-%s-set'%(name), zoom)
    print()
    print()
      
    
    if mqi:
      S = simplify_set(g, lgc.flow_clustering(g, Si, method='mqi')[0])
      result['mqi'] = S
      print("MQI set: ")
      show_set_figures(g, S, 'usroads-simplify-%s-mqi'%(name), zoom)
      print()
      print()
    
    if fi:
      S = simplify_set(g, lgc.flow_clustering(g, Si, method='sl', delta=0)[0])
      result['fi'] = S
      print("FI set: ")
      show_set_figures(g, S, 'usroads-simplify-%s-fi'%(name), zoom)
      print()
      print()
   
    if sl01:
      S = simplify_set(g, lgc.flow_clustering(g, Si, method='sl', delta=0.1)[0])
      result['sl01'] = S
      print("SL01 set: ")
      show_set_figures(g, S, 'usroads-simplify-%s-sl01'%(name), zoom)
      print()
      print()
      
    if sl05:
      S = simplify_set(g, lgc.flow_clustering(g, Si, method='sl', delta=0.5)[0])
      result['sl05'] = S
      print("SL05 set: ")
      show_set_figures(g, S, 'usroads-simplify-%s-sl05'%(name), zoom)
      print()
      print()
      
     
    if sl1:
      S = simplify_set(g, lgc.flow_clustering(g, Si, method='sl', delta=1)[0])
      result['sl1'] = S
      print("SL1 set: ")
      show_set_figures(g, S, 'usroads-simplify-%s-sl1'%(name), zoom)
      print()
      print()
      
    return result

In [ ]:
# this uses the horizontal S1 set above and is for testing subroutines
drawing, bm, ax = drawusmap()
myhighlight(drawing, S1small)
#zoomfig(g, S1small, bm, ax, drawing)
show_set_figures(g, S1small, "figtest", True)

In [ ]:
# This was used for testing some of the options 
#Shoriz = [i for i in range(g._num_vertices) if xy[i,1] >= 39]
#Shoriz_results = study_simple_set(g, Shoriz, 'horiz', fi=False, sl05=True, sl1=True, sl01=False)

In [ ]:
Shoriz = [i for i in range(g._num_vertices) if xy[i,1] >= 39]
Shoriz_results = study_simple_set(g, Shoriz, 'horiz')

In [ ]:
Svert = [i for i in range(g._num_vertices) if xy[i,0] >= -85]
Svert_results = study_simple_set(g, Svert, 'vert')

In [ ]:
Svert85 = [i for i in range(g._num_vertices) if xy[i,0] >= -85]
drawing, bm, ax = drawusmap()
myhighlight(drawing, Svert85)  
g.set_scores(Svert85)
Svert_results = study_simple_set(g, Svert85, 'vert85')

In [ ]:
Svert88 = [i for i in range(g._num_vertices) if xy[i,0] >= -88]
drawing, bm, ax = drawusmap()
myhighlight(drawing, Svert88)  
g.set_scores(Svert88)
Svert88_results = study_simple_set(g, Svert88, 'vert88')

In [ ]:
Scenter = [i for i in range(g._num_vertices) if np.sqrt((xy[i,1] - 38.742475)**2 + (xy[i,0]--104.848444)**2) <= 10]
Scenter_results = study_simple_set(g, Scenter, 'center')

## Identify captials and align

In [ ]:
capitals = """Name: Alabama

Capital Name: Montgomery

Capital Latitude: 32.361538

Capital Longitude: -86.279118

Name: Arizona

Capital Name: Phoenix

Capital Latitude: 33.448457

Capital Longitude: -112.073844

Name: Arkansas

Capital Name: Little Rock

Capital Latitude: 34.736009

Capital Longitude: -92.331122

Name: California

Capital Name: Sacramento

Capital Latitude: 38.555605

Capital Longitude: -121.468926

Name: Colorado

Capital Name: Denver

Capital Latitude: 39.7391667

Capital Longitude: -104.984167

Name: Connecticut

Capital Name: Hartford

Capital Latitude: 41.767

Capital Longitude: -72.677

Name: Delaware

Capital Name: Dover

Capital Latitude: 39.161921

Capital Longitude: -75.526755

Name: Florida

Capital Name: Tallahassee

Capital Latitude: 30.4518

Capital Longitude: -84.27277

Name: Georgia

Capital Name: Atlanta

Capital Latitude: 33.76

Capital Longitude: -84.39

Name: Idaho

Capital Name: Boise

Capital Latitude: 43.613739

Capital Longitude: -116.237651

Name: Illinois

Capital Name: Springfield

Capital Latitude: 39.783250

Capital Longitude: -89.650373

Name: Indiana

Capital Name: Indianapolis

Capital Latitude: 39.790942

Capital Longitude: -86.147685

Name: Iowa

Capital Name: Des Moines

Capital Latitude: 41.590939

Capital Longitude: -93.620866

Name: Kansas

Capital Name: Topeka

Capital Latitude: 39.04

Capital Longitude: -95.69

Name: Kentucky

Capital Name: Frankfort

Capital Latitude: 38.197274

Capital Longitude: -84.86311

Name: Louisiana

Capital Name: Baton Rouge

Capital Latitude: 30.45809

Capital Longitude: -91.140229

Name: Maine

Capital Name: Augusta

Capital Latitude: 44.323535

Capital Longitude: -69.765261

Name: Maryland

Capital Name: Annapolis

Capital Latitude: 38.972945

Capital Longitude: -76.501157

Name: Massachusetts

Capital Name: Boston

Capital Latitude: 42.2352

Capital Longitude: -71.0275

Name: Michigan

Capital Name: Lansing

Capital Latitude: 42.7335

Capital Longitude: -84.5467

Name: Minnesota

Capital Name: Saint Paul

Capital Latitude: 44.95

Capital Longitude: -93.094

Name: Mississippi

Capital Name: Jackson

Capital Latitude: 32.320

Capital Longitude: -90.207

Name: Missouri

Capital Name: Jefferson City

Capital Latitude: 38.572954

Capital Longitude: -92.189283

Name: Montana

Capital Name: Helana

Capital Latitude: 46.595805

Capital Longitude: -112.027031

Name: Nebraska

Capital Name: Lincoln

Capital Latitude: 40.809868

Capital Longitude: -96.675345

Name: Nevada

Capital Name: Carson City

Capital Latitude: 39.160949

Capital Longitude: -119.753877

Name: New Hampshire

Capital Name: Concord

Capital Latitude: 43.220093

Capital Longitude: -71.549127

Name: New Jersey

Capital Name: Trenton

Capital Latitude: 40.221741

Capital Longitude: -74.756138

Name: New Mexico

Capital Name: Santa Fe

Capital Latitude: 35.667231

Capital Longitude: -105.964575

Name: New York

Capital Name: Albany

Capital Latitude: 42.659829

Capital Longitude: -73.781339

Name: North Carolina

Capital Name: Raleigh

Capital Latitude: 35.771

Capital Longitude: -78.638

Name: North Dakota

Capital Name: Bismarck

Capital Latitude: 48.813343

Capital Longitude: -100.779004

Name: Ohio

Capital Name: Columbus

Capital Latitude: 39.962245

Capital Longitude: -83.000647

Name: Oklahoma

Capital Name: Oklahoma City

Capital Latitude: 35.482309

Capital Longitude: -97.534994

Name: Oregon

Capital Name: Salem

Capital Latitude: 44.931109

Capital Longitude: -123.029159

Name: Pennsylvania

Capital Name: Harrisburg

Capital Latitude: 40.269789

Capital Longitude: -76.875613

Name: Rhode Island

Capital Name: Providence

Capital Latitude: 41.82355

Capital Longitude: -71.422132

Name: South Carolina

Capital Name: Columbia

Capital Latitude: 34.000

Capital Longitude: -81.035

Name: South Dakota

Capital Name: Pierre

Capital Latitude: 44.367966

Capital Longitude: -100.336378

Name: Tennessee

Capital Name: Nashville

Capital Latitude: 36.165

Capital Longitude: -86.784

Name: Texas

Capital Name: Austin

Capital Latitude: 30.266667

Capital Longitude: -97.75

Name: Utah

Capital Name: Salt Lake City

Capital Latitude: 40.7547

Capital Longitude: -111.892622

Name: Vermont

Capital Name: Montpelier

Capital Latitude: 44.26639

Capital Longitude: -72.57194

Name: Virginia

Capital Name: Richmond

Capital Latitude: 37.54

Capital Longitude: -77.46

Name: Washington

Capital Name: Olympia

Capital Latitude: 47.042418

Capital Longitude: -122.893077

Name: West Virginia

Capital Name: Charleston

Capital Latitude: 38.349497

Capital Longitude: -81.633294

Name: Wisconsin

Capital Name: Madison

Capital Latitude: 43.074722

Capital Longitude: -89.384444

Name: Wyoming

Capital Name: Cheyenne

Capital Latitude: 41.145548

Capital Longitude: -104.802042""".split("\n\n")
caps=[]
for i in range(0,len(capitals),4):
  xc = float(capitals[i+2].split(": ")[1])
  yc = float(capitals[i+3].split(": ")[1])
  xyc = np.array([yc,xc])
  dists = np.sum((xy - xyc)**2,axis=1) 
  node = np.sum((xy - xyc)**2,axis=1).argmin()
  caps.append((capitals[i].split(": ")[1], node))
  
def random_walk(g,start,k):
    curr_node = start
    iters = 0
    trajectory = [curr_node]
    while iters < k:
        neighs = g.aj[g.ai[curr_node]:g.ai[curr_node+1]]
        curr_node = np.random.choice(neighs)
        trajectory.append(curr_node)
        iters += 1
    return trajectory  
  
def cap_random_walk_sets(k=1000):
  # Form random walks   
  np.random.seed(1)
  caprw = set()
  for cap in caps:
    capnode = cap[1] 
    S = random_walk(g,capnode,k)
    caprw.update(S)
    #print(cap[0], " cond=", capcond)
  return list(caprw)    
caprws = {}
caprws[1000] = cap_random_walk_sets(1000)
caprws[500] = cap_random_walk_sets(500)
caprws[250] = cap_random_walk_sets(250)
caprws[100] = cap_random_walk_sets(100)

# Form shortest path sets
import networkx as nx
nG = nx.from_scipy_sparse_matrix(g.adjacency_matrix)
spsets = set()
for cap in caps:
  capnode = cap[1] 
  for cap2 in caps:
    capnode2 = cap2[1]
    if capnode2 > capnode:
      spsets.update(nx.shortest_path(nG, source=capnode, target=capnode2))
spsets = list(spsets)


In [ ]:
spsets_results = study_simple_set(g, spsets, 'sp')

In [ ]:
caprws_results = {}
for rwlen in caprws:
  caprws_results[rwlen] = study_simple_set(g, caprws[rwlen], 'rw%i'%(rwlen))

Try a random walk around Indianapolis. 

In [ ]:
def cap_study(node, rwlen, nwalks, name, offset=0.75):
  np.random.seed(0)
  Scap = set()
  for i in range(nwalks):
    Scap.update(random_walk(g, node, rwlen))
  Scap = list(Scap)  
  drawing, bm, ax = drawusmap()
  myhighlight(drawing, Scap)
  zoomfig(g, Scap, bm, ax, drawing, offset=offset)
  bigger_graph(drawing)
  pprint.pprint(g.set_scores(Scap))
  plt.savefig('usroads-simplify-rwcap-%s.png'%(name), dpi=200,bbox_inches='tight',pad_inches=0)
  
  for delta in [1, 0.5, 0.1]:
    Sfi = lgc.flow_clustering(g, Scap, method="sl", delta=delta)[0]
    drawing, bm, ax = drawusmap()
    myhighlight(drawing, Sfi)
    zoomfig(g, Sfi, bm, ax, drawing, offset=offset)
    bigger_graph(drawing)
    print("Delta = %f"%(delta))
    pprint.pprint(g.set_scores(Sfi))
    plt.savefig('usroads-simplify-rwcap-%s-sl%s.png'%(name,("%.1f"%(delta)).replace(".","")), 
                    dpi=200,bbox_inches='tight',pad_inches=0)

In [ ]:
# Indiana
#cap_study(caps[11][1], 35, 100, "ind") # this one is pretty good
#cap_study(caps[11][1], 40, 120, "ind") # this is probably better
#cap_study(caps[11][1], 60, 200, "ind") # see final below 

In [ ]:
# Virginia
#cap_study(caps[43][1], 50, 150, "vir") # pretty good... 
#cap_study(caps[43][1], 60, 200, "vir") # these are GREAT! we are going to use these... 
# see the final study below. 

In [ ]:
# Finalize the Virginia figs
def cap_study_final(node, rwlen, nwalks, name, offset=0.35):
  np.random.seed(0)
  Scap = set()
  for i in range(nwalks):
    Scap.update(random_walk(g, node, rwlen))
  Scap = list(Scap)  
  
  # it's fast enough just to compute these... 
  Sall = set(Scap)
  for delta in [float('inf'), 1.5, 1, 0.5, 0.1]:
    if delta == float('inf'):
      Sall.update(lgc.flow_clustering(g, Scap, method="mqi")[0])
    else:
      Sall.update(lgc.flow_clustering(g, Scap, method="sl", delta=delta)[0])
  Sall = list(Sall)
  
  drawing, bm, ax = drawusmap()
  myhighlight(drawing, Scap)
  zoomfig(g, Sall, bm, ax, drawing, offset=offset)
  bigger_graph(drawing)
  pprint.pprint(g.set_scores(Scap))
  plt.savefig('usroads-simplify-rwcap-%s.png'%(name), dpi=200,bbox_inches='tight',pad_inches=0)
  
  for delta in [float('inf'), 1.5, 1, 0.5, 0.1]:
    if delta == float('inf'):
      Sfi = lgc.flow_clustering(g, Scap, method="mqi")[0]
    else:
      Sfi = lgc.flow_clustering(g, Scap, method="sl", delta=delta)[0]
    drawing, bm, ax = drawusmap()
    myhighlight(drawing, Sfi)
    zoomfig(g, Sall, bm, ax, drawing, offset=offset)
    bigger_graph(drawing)
    print("Delta = %f"%(delta))
    pprint.pprint(g.set_scores(Sfi))
    plt.savefig('usroads-simplify-rwcap-%s-sl%s.png'%(name,("%.1f"%(delta)).replace(".","")), 
                    dpi=200,bbox_inches='tight',pad_inches=0)
cap_study_final(caps[43][1], 60, 200, "vir")  

In [ ]:
# California
# cap_study(caps[3][1], 35, 100, "calif") this one is okay ... 
#cap_study(caps[3][1], 40, 120, "calif")
#cap_study(caps[3][1], 60, 200, "calif")
cap_study_final(caps[3][1], 60, 200, "calif")  

In [ ]:
# make the set of Indiana figures
cap_study_final(caps[11][1], 60, 200, "ind")  

## These are extra codes below. 

In [ ]:
cap_indiana = caps[11][1]
np.random.seed(0)
Scap = set()
for i in range(100):
    Scap.update(random_walk(g, cap_indiana, 20))
Scap = list(Scap)  
drawing, bm, ax = drawusmap()
myhighlight(drawing, Scap)
zoomfig(g, Scap, bm, ax, drawing, offset=0.75)
g.set_scores(Scap)

In [ ]:
Sfi = lgc.flow_clustering(g, Scap, method="sl", delta=0.0)[0]
#drawing, bm, ax = drawusmap()
#myhighlight(drawing, Sfi)
#zoomfig(g, Sfi, bm, ax, drawing, offset=0.75)
g.set_scores(Sfi)

In [ ]:
## Try the same thing for Capital of Virginia and Capital of California... 


In [ ]:
cap_calif = caps[3][1]
Scap = set()
for i in range(100):
    Scap.update(random_walk(g, cap_calif, 20))
Scap = list(Scap)  
drawing, bm, ax = drawusmap()
myhighlight(drawing, Scap)
zoomfig(g, Scap, bm, ax, drawing, offset=0.75)
bigger_graph(
plt.savefig('usroads-simplify-network-1.png', dpi=200,bbox_inches='tight',pad_inches=0)
g.set_scores(Scap)

In [ ]:
Sfi = lgc.flow_clustering(g, Scap, method="sl", delta=0.0)[0]
drawing, bm, ax = drawusmap()
myhighlight(drawing, Sfi)
zoomfig(g, Sfi, bm, ax, drawing, offset=0.75)
g.set_scores(Sfi)

## Metis partitions

In [ ]:
import metis
import networkx as nx
edgecuts, parts = metis.part_graph(nG,2)
nG = nx.from_scipy_sparse_matrix(g.adjacency_matrix)
P1 = [i for i in range(g._num_vertices) if parts[i] == 0]
P = simplify_set(g,P1)
g.set_scores(P)

In [ ]:
Sf = lgc.flow_clustering(g, P, method='mqi')[0]
g.set_scores(Sf)

In [ ]:
Sf = lgc.flow_clustering(g, P, method='sl', delta=0.1)[0]
g.set_scores(Sf)

In [ ]:
Sf = lgc.flow_clustering(g, P, method='sl', delta=0.0)[0]
g.set_scores(Sf)

In [ ]:
g.set_scores(Sf)